# RFM Analizi ile Müşteri Segmentasyonu (Customer Segmentation with RFM)

## İş Problemi (Business Problem)

Online ayakkabı mağazası olan FLO müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama
stratejileri belirlemek istiyor. Buna yönelik olarak müşterilerin davranışları tanımlanacak ve bu
davranışlardaki öbeklenmelere göre gruplar oluşturulacak.

## Veri Seti Hikayesi

Veri seti Flo’dan son alışverişlerini 2020 - 2021 yıllarında OmniChannel (hem online hem offline alışveriş yapan)
olarak yapan müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.

### 📊 Veri Seti Değişkenleri

| Değişken Adı                         | Açıklama                                                                 |
|-------------------------------------|--------------------------------------------------------------------------|
| master_id                           | Eşsiz müşteri numarası                                                  |
| order_channel                       | Alışveriş yapılan platform (Android, iOS, Desktop, Mobile)             |
| last_order_channel                  | En son alışveriş yapılan kanal                                          |
| first_order_date                    | Müşterinin yaptığı ilk alışveriş tarihi                                |
| last_order_date                     | Müşterinin yaptığı son alışveriş tarihi                                |
| last_order_date_online              | Müşterinin online platformda yaptığı son alışveriş tarihi              |
| last_order_date_offline             | Müşterinin offline platformda yaptığı son alışveriş tarihi             |
| order_num_total_ever_online         | Online platformdaki toplam alışveriş sayısı                            |
| order_num_total_ever_offline        | Offline platformdaki toplam alışveriş sayısı                           |
| customer_value_total_ever_offline   | Offline alışverişlerde ödenen toplam ücret                             |
| customer_value_total_ever_online    | Online alışverişlerde ödenen toplam ücret                              |
| interested_in_categories_12         | Son 12 ayda alışveriş yapılan kategoriler                              |


## Ön Ayarlar

In [ ]:
import datetime as dt
import pandas as pd

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## 1. Veriyi Anlama (Data Understanding) ve Hazırlama

In [ ]:
# veri setini okuma ve kopyasını oluşturma 

df_ = pd.read_csv('/kaggle/input/flo-data/flo_data_20k.csv')
df = df_.copy()

In [ ]:
# Veri setinde
# a. İlk 10 gözlem
df.head(10)

# b. Değişken isimleri
df.columns

# c. Betimsel istatistik
df.describe().T

# d. Boş değer incelemesi
df.isnull().sum()
                     
# e. Değişken tipleri incelemesi
df.info()

In [ ]:
# Omnichannel müşterilerin hem online'dan hem de offline platformlardan alışveriş yaptığını ifade etmektedir. 
# Herbir müşterinin toplam alışveriş sayısı ve harcaması için yeni değişkenlerin oluşturulması
df["total_order_num"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["total_customer_value"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [ ]:
# Tarih ifade eden değişkenlerin tipinin date'e çevrilmesi
date_columns = [col for col in df.columns if "date" in col]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

In [ ]:
# Alışveriş kanallarındaki müşteri sayısının, toplam alınan ürün sayısının ve toplam harcamaların dağılımının incelenmesi
df.groupby("order_channel").agg({"master_id": "count",
                                 "total_order_num": "sum",
                                 "total_customer_value": "sum"})

In [ ]:
# En fazla kazancı getiren ilk 10 müşterinin sıralanması
df.groupby("master_id").agg({"total_customer_value": "sum"}).sort_values("total_customer_value", ascending=False)[:10]

In [ ]:
# En fazla siparişi veren ilk 10 müşterinin sıralanması
df.groupby("master_id").agg({"total_order_num": "sum"}).sort_values("total_order_num", ascending=False)[:10]

## Veri Ön Hazırlık Sürecinin Fonksiyonlaştırılması

In [ ]:
def data_preparation(dataframe):

    dataframe["total_order_num"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["total_customer_value"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    dataframe.dropna(inplace=True)
    date_columns = [col for col in dataframe.columns if "date" in col]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)

## 2. RFM Metriklerinin Hesaplanması

In [ ]:
# Analiz günü (son sipariş tarihinden 2 gün sonrası olarak seçilmiştir)
df["last_order_date"].max()
today_date = dt.datetime(2021, 6, 1)

In [ ]:
# Recency, Frequency, Monetary
rfm = df.groupby("master_id").agg({"last_order_date": lambda last_order_date : (today_date - last_order_date.max()).days,
                                   "total_order_num": lambda total_order_num: total_order_num.sum(),
                                   "total_customer_value": lambda total_customer_value: total_customer_value.sum()})

rfm.columns = ['recency', 'frequency', 'monetary']

## 3. RFM ve RF Skorlarının Hesaplanması

In [ ]:
rfm['recency_score'] = pd.qcut(rfm['recency'], 5, labels = [5, 4, 3, 2, 1])

rfm['frequency_score'] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels = [1, 2, 3, 4, 5])

rfm['monetary_score'] = pd.qcut(rfm['monetary'], 5, labels = [1, 2, 3, 4, 5])

rfm['RF_SCORE'] = (rfm['recency_score'].astype(str) +
                   rfm['frequency_score'].astype(str))

## 4. RF Skorlarının Segment Olarak Tanımlanması

In [ ]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex = True)

In [ ]:
# Segmentlerin recency, frequnecy ve monetary ortalamalarının incelenmesi
rfm[['segment', 'recency', 'frequency', 'monetary']].groupby('segment').agg(["mean", "count"])

## 5. RFM Analizi Yardımı ile İlgili Profildeki Müşterilerin Bulunması ve Müşteri id'lerinin csv'ye Kaydedilmesi

In [ ]:
# a. FLO bünyesine yeni bir kadın ayakkabı markası dahil ediyor. Dahil ettiği markanın ürün fiyatları genel müşteri tercihlerinin üstünde. Bu nedenle markanın
# tanıtımı ve ürün satışları için ilgilenecek profildeki müşterilerle özel olarak iletişime geçeilmek isteniliyor. Bu müşterilerin sadık (champions,loyal_customers) ve
# kadın kategorisinden alışveriş yapan kişiler olması planlandı. Müşterilerin id numaralarını csv dosyasına yeni_marka_hedef_müşteri_id.cvs
# olarak kaydedilmesi

yeni_marka_hedef_müşteri_segmenti = rfm[rfm['segment'].isin(['champions', 'loyal_customers'])]
yeni_marka_hedef_müşteri = df[df['master_id'].isin(yeni_marka_hedef_müşteri_segmenti.index) &
                           df['interested_in_categories_12'].str.contains("KADIN")]

yeni_marka_hedef_müşteri['master_id'].to_csv("yeni_marka_hedef_müşteri_id.csv", index=False)

In [ ]:
# b. Erkek ve Çoçuk ürünlerinde %40'a yakın indirim planlanmaktadır. Bu indirimle ilgili kategorilerle ilgilenen geçmişte iyi müşteri olan ama uzun süredir
# alışveriş yapmayan kaybedilmemesi gereken müşteriler, uykuda olanlar ve yeni gelen müşteriler özel olarak hedef alınmak isteniliyor. Uygun profildeki müşterilerin id'lerini csv dosyasına indirim_hedef_müşteri_ids.csv
# olarak kaydedilmesi

kampanya_hedef_musteri_segmenti = rfm[rfm['segment'].isin(['cant_loose', 'hibernating', 'new_customers'])]
kampanya_hedef_musteri = df[df['master_id'].isin(kampanya_hedef_musteri_segmenti).index &
                         df['interested_in_categories_12'].str.contains("ERKEK", "COCUK")]

kampanya_hedef_musteri['master_id'].to_csv("kampanya_40_hedef_musteri_id.csv", index=False)

## 6. Tüm Sürecin Fonksiyonlaştırılması

In [ ]:
def create_rfm(dataframe, csv=False):

    # data preparetion
    dataframe["total_order_num"] = (dataframe["order_num_total_ever_online"] +
                                    dataframe["order_num_total_ever_offline"])
    dataframe["total_customer_value"] = (dataframe["customer_value_total_ever_offline"] +
                                         dataframe["customer_value_total_ever_online"])
    dataframe.dropna(inplace=True)
    date_columns = [col for col in dataframe.columns if "date" in col]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)

    # calculating RFM metrics
    dataframe["last_order_date"].max()
    today_date = dt.datetime(2021, 6, 1)
    rfm = dataframe.groupby("master_id").agg(
        {"last_order_date": lambda last_order_date: (today_date - last_order_date.max()).days,
         "total_order_num": lambda total_order_num: total_order_num.sum(),
         "total_customer_value": lambda total_customer_value: total_customer_value.sum()})
    rfm.columns = ['recency', 'frequency', 'monetary']

    # calculating RFM scores
    rfm['recency_score'] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm['frequency_score'] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm['monetary_score'] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])
    rfm['RF_SCORE'] = (rfm['recency_score'].astype(str) +
                       rfm['frequency_score'].astype(str))

    #creating & analysing RFM segments
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_Risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

    rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)

    rfm = rfm[['recency', 'frequency', 'monetary', 'segment']]

    if csv:
        rfm.to_csv("rfm.csv")

    return rfm

In [ ]:
df = df_.copy()

rfm_new = create_rfm(df, csv=True)